In [63]:
from sqlalchemy import create_engine
import boto3
import botocore
import json
import os
from test_config import *
import sys
import flask
from flask import Flask, request
from flask_cors import CORS

In [72]:
DB_URL = 'postgresql+psycopg2://{user}:{pw}@{url}/{db}'.format(
    user=POSTGRES_USER, pw=POSTGRES_PW, url=POSTGRES_URL, db=POSTGRES_DB)
db = create_engine(DB_URL)

if sys.version_info.major == 2:
    reload(sys)
    sys.setdefaultencoding('UTF8')
app = Flask(__name__)
CORS(app)

In [73]:
def generate_query(response_data):
    query=""
    stack_id=response_data.get('stack_id')
    recommendation_type=response_data.get('recommendation_type')
    package_name = response_data.get('package_name')
    feedback_type=response_data.get('feedback_type')
    if is_valid(stack_id) and is_valid(recommendation_type) and is_valid(package_name) and is_valid(feedback_type):
        query="insert into feedback (stack_id, recommendation_type, package_name, feedback_type)"
        query+="values('{}','{}','{}',{});".format(stack_id, recommendation_type, package_name, feedback_type)
    return query 

def execute(query):
    try:
        if query=="":
            return {"status":"failure","message":"Empty query"}
        else:
            db.execute(query)
            return {"status":"success","message":"All good"}
    except Exception as e:
        return {"status":"failure", "message":"{}".format(e)}
def processing(input_json):
    return execute(generate_query(input_json))

def is_valid(param_value):
    return param_value is not None

In [74]:
@app.route('/')
def heart_beat():
    return flask.jsonify({"status": "ok"})
@app.route('/api/v1/submit_feedback', methods=['POST'])
def get_path():
    input_json = request.get_json()
    response= processing(input_json)
    return flask.jsonify(response)

In [ ]:
app.run(port=8080)